## Statement:
You are embarking on an expedition in the remote valleys of Papua New Guinea, one of the last
uncharted regions on Earth. During your journey, you encounter a tribe that operates on a barter
system instead of using money. In this system, there are N different commodities available for
trade, and the exchange rates between these commodities are defined in a two-dimensional matrix.
For instance, you might find that three sheep can be traded for seven goats, or four goats can be
exchanged for 200 pounds of wheat, and so on.

Your task is to develop an efficient algorithm to determine whether it is possible to exploit arbitrage
opportunities within this trading system. Arbitrage, in this context, means finding a way to start
with a single unit of a particular commodity (let's call it commodity 0) and convert it back into
more than one unit of commodity 0 by following a sequence of exchanges. You should assume that
there are no transaction costs, the exchange rates are constant, and you can trade fractional
quantities of items.

<hr>

### Consider the below matrix depicting the exchange rates between different commodities:

$$
\left(\begin{array}{cc}
A&B&C\\
1&2&1/3\\
1/2&1&2\\
3&1/2&1\\
\end{array}\right)
$$

In the matrix please notice that trading rate between two commodities is consistent i.e., **x** commodity of A is traded for **y** commondity of B also, **y** B will be traded for **x** A. So the trading rate is consistent with the opposite direction too of a trade among two commodities.

Aribtrage scenario:

* **1** commodity of A can be traded for **2** from B.
* **2** B can be traded for **4** C (2*2).
* Finally, **4** C can be traded for **12** A (4*3).

So we were able to trade 1 A to finally obtain 12 A through a series of trades involving different commodities, each being exchanging in their respective trade rate.


## Approach that utilizing depth first search (DFS) ⬇️

Where we view each commodity as a node in our graph and we begin traversal from our target commodity. This is a fully-connected graph where each commodity can be traded to another commodity following the exchange rate.

We start with a target commodity present at a particular quantity, we want to know if we can **establish an arbitrage** on that target commodity.

* In each trade we traverse from one commodity to another
* After a trade we check what is the value of the received good in terms of our target commodity and if greater than initial quantity -> "Arbitrage"
* Else we perform more trades and identify if possible.
* If not possible, we terminate

In [1]:
from typing import List

In [2]:
trade_rate = [
    [1, 2, 1/3],
    [1/2, 1, 2],
    [3, 1/2, 1],
]
# We are interested in arbitrage of commodity A, denoted by index '0'
target_commodity, start_quantity = 0, 1
trades = []

In [3]:
def has_arbitrage(i, curr: int, target_commodity: int, quantity: int):
    """Checks if arbitrage is possible! Follows a depth first search approach to find the first trade route to establish an arbitrage.

    Args:
        curr (int): The current item that we are considering after previous trade to this 'curr' item
        target_commodity (int): The item of interest 'target_commodity'
        quantity (int): The current quantity of items we have, measured in terms of our item of interest (target_commodity)

    Returns:
        bool: Whether arbitrage is possible
    """
    N = len(trade_rate)
    if (i == N):
        return False

    for next_comm in range(N):
        if (next_comm == curr or next_comm == target_commodity):
            continue
        # Trade rate from the current item curr to item 'next_comm'
        exchange_rate = trade_rate[curr][next_comm]
        curr_to_next_comm_qty = exchange_rate*quantity  # Quantity in item 'next_comm'
        # Quantity of item 'next_comm' if exchanged for our target item
        next_comm_to_target_qty = curr_to_next_comm_qty * \
            trade_rate[next_comm][target_commodity]

        # If trades for more than STARTING target quantity then arbitrage is possible.
        if (next_comm_to_target_qty > start_quantity):
            trades.append((curr, next_comm, curr_to_next_comm_qty,
                          next_comm_to_target_qty))
            return True
        else:
            if (has_arbitrage(i+1, next_comm, target_commodity, curr_to_next_comm_qty)):
                trades.append((curr, next_comm, curr_to_next_comm_qty,
                               next_comm_to_target_qty))
                return True

    return False

In [7]:
# Scenario when arbitrage possible

trade_rate = [
    [1, 2, 1/3],
    [1/2, 1, 2],
    [3, 1/2, 1],
]

status = has_arbitrage(0, target_commodity, target_commodity, 1)

if (status):
    N = len(trades)
    print(f"Target Commodity {chr(target_commodity + 65)}, Quantity: {start_quantity}")
    for i, ele in enumerate(reversed(trades)):
        print(f"{chr(ele[0] + 65)} -> {chr(ele[1] + 65)}, Exchange: {ele[2]}")
        
        if (i == N - 1):
        # This is the last trade, where we trade to our target commodity
            end_quantity = ele[3]
            print(
                f">> FINAL - {chr(ele[1] + 65)} -> {chr(target_commodity + 65)}: {ele[3]}")
else:
    print("Couldn't establish arbitrage")

Target Commodity A, Quantity: 1
A -> B, Exchange: 2
B -> C, Exchange: 4
A -> B, Exchange: 2
B -> C, Exchange: 4
>> FINAL - C -> A: 12


In [8]:
# Scenario when arbitrage not possible

trade_rate = [
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1],
]
status = has_arbitrage(0, target_commodity, target_commodity, 1)

if (status):
    N = len(trades)
    print(f"Target Commodity {chr(target_commodity + 65)}, Quantity: {start_quantity}")
    for i, ele in enumerate(reversed(trades)):
        print(f"{chr(ele[0] + 65)} -> {chr(ele[1] + 65)}, Exchange: {ele[2]}")
        
        if (i == N - 1):
        # This is the last trade, where we trade to our target commodity
            end_quantity = ele[3]
            print(
                f">> FINAL - {chr(ele[1] + 65)} -> {chr(target_commodity + 65)}: {ele[3]}")
else:
    print("Couldn't establish arbitrage")


Couldn't establish arbitrage


## Analysis

Time Complexity: $O(N^2)$ (You have N recursive calls and each call you check N vertices = $N^2$)

Space Complexity: $O(N)$ (The stack depth is the number of vertices which is N)